In [1]:
print("hello")

hello


In [2]:
%pwd

'd:\\projects\\medical-chatbot\\research'

In [ ]:

import os
os.chdir("../")

In [6]:
%pwd

'd:\\projects\\medical-chatbot'

In [7]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# load data

In [8]:
def extract_pdf_files(data_path):
    # here in data folder you can add more pdfs too, loader will load all of them
    loader=DirectoryLoader(
        data_path,glob="*.pdf",loader_cls=PyPDFLoader
    )
    
    documents=loader.load()
    return documents


In [16]:
extracted_data=extract_pdf_files("data")
# extracted_data
# langchain returning the document format

In [17]:
len(extracted_data) #number of pages

637

In [18]:
# page content is needed and it is present in document that we get
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs:List[Document])->List[Document]:
    """
    Given a list of documents, i wnat to return a new list of documents that only contains the source and the page_content
    """
    
    minimal_docs:List[Document]=[]
    for doc in docs:
        src=doc.metadata.get('source')
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source":src}
            )
        )
    
    return minimal_docs



In [19]:
extracted_text=filter_to_minimal_docs(extracted_data)

# chunkning

In [22]:
def text_splitter(extracted_text):
    # here create the chunks of the extracted data
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
        # length_function=len
    )
    
    chunked_text=text_splitter.split_documents(extracted_text)
    return chunked_text

In [24]:
len(text_splitter(extracted_text))
texts_chunk=text_splitter(extracted_text)

# do embeddings of the chunks 

In [37]:
# hugging face embedding model
from langchain.embeddings import HuggingFaceEmbeddings

# download the pretrained embedding model we want to use

def download_embedding_model():
    """
    Download the model and return the hugging face embedddings
    """
    model_name="sentence-transformers/all-MiniLM-L6-v2"
    embeddings=HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

In [38]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [39]:
# access the pinecone and open ai api key  
PINECONE_API_KEY=os.getenv('PINECONE_API_KEY')
OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')

os.environ['PINECONE_API_KEY']=PINECONE_API_KEY
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY